#<h1><font color="#110068" size=5> TFE</font></h1>


<h1><font color="#110068" size=6>Modelo de redes convolucionales para la detección de Patologías oculares mediante el procesamiento de imágenes </font></h1>

<div style="text-align: right">
<font size=3> </font><br>
ALUMNO 1: Margarita Cardenas  </font><br>
ALUMNO 2: Miguel Rojas </font><br>
ALUMNO 3: Walter Vega </font><br>

</font><br>
<font size=3>TFE:
Implementar una Inteligencia Artificial para la detección de patologías oculares mediante el procesamiento de imágenes
</font><br>
<font size=3>UNIR</font>
</div>

# Instalación de complementos y librerias necesarias

In [ ]:
!pip install pydrive2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 649.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.8.0
    Uninstalling google-api-python-client-1.8.0:
      Successfully uninstalled google-api-python-client-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2

## Descargamos el dataset de imágenes (Desde una carpeta compartida en GoogleDrive) y descomprimimos el conjunto de imágenes con los que vamos a trabajar

In [ ]:
import os

from pydrive2.auth import GoogleAuth
from google.colab import drive as colab_drive
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Autenticar el usuario
auth.authenticate_user()

# Inicializar GoogleAuth y establecer las credenciales
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# Crear la instancia de GoogleDrive
drive = GoogleDrive(gauth)

file_id = '1KXCIri7VEdjwCnTPocehAf73z_7jgbgF'

if (not os.path.exists("/content/DS_TFE_final.zip")):
  download = drive.CreateFile({'id': file_id})
  # Download the file to a local disc
  download.GetContentFile('DS_TFE_final.zip')
  !unzip "/content/DS_TFE_final.zip"



Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR273.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR274.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR275.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR277.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR278.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR280.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR281.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR282.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR283.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR286.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR288.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR289.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR29.jpg  
  inflating: DS_TFE_Final/train/Diabetic_Retinopathy/DR290.jpg  
  inflating: DS_TFE_Final/trai

## Importación de librerías necesarias para el notebook

In [ ]:
import pandas
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn
import cv2

import glob
import re

# Modelo YOLO 1

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
model = YOLO('yolov8n-cls.pt') # load a pretrained model (recommended for training)

# Train the model
model.train(data='/content/DS_TFE_Final', epochs=5)

Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/DS_TFE_Final, epochs=5, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning /content/DS_TFE_Final/train... 5432 images, 0 corrupt: 100%|██████████| 5432/5432 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/DS_TFE_Final/val... 775 images, 0 corrupt: 100%|██████████| 775/775 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train5
Starting training for 5 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 25/25 [00:42<00:00,  1.68s/it]

                   all      0.748      0.995


...

                   all       0.81          1



5 epochs completed in 0.516 hours.
Optimizer stripped from runs/classify/train5/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train5/weights/best.pt, 3.0MB

Validating runs/classify/train5/weights/best.pt...
Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1442566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 25/25 [00:47<00:00,  1.89s/it]


                   all      0.808          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5
Results saved to runs/classify/train5


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9e7d762170>
curves: []
curves_results: []
fitness: 0.9038709700107574
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8077419400215149, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9038709700107574}
save_dir: PosixPath('runs/classify/train5')
speed: {'preprocess': 0.08069069154800908, 'inference': 0.3438380456739857, 'loss': 0.0005155994046118951, 'postprocess': 0.000488219722624748}
task: 'classify'
top1: 0.8077419400215149
top5: 1.0

In [ ]:
results_val = model.val()

print(results_val)

Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1442566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


val: Scanning /content/DS_TFE_Final/val... 775 images, 0 corrupt: 100%|██████████| 775/775 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 49/49 [00:48<00:00,  1.01it/s]


                   all       0.81          1
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train52


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9d20454ca0>
curves: []
curves_results: []
fitness: 0.9051612913608551
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8103225827217102, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9051612913608551}
save_dir: PosixPath('runs/classify/train52')
speed: {'preprocess': 0.10087690045756678, 'inference': 0.6766623835409841, 'loss': 0.0009358313775831654, 'postprocess': 0.003566741943359375}
task: 'classify'
top1: 0.8103225827217102
top5: 1.0


In [ ]:
model2 = YOLO('yolov8n-cls.pt') # load a pretrained model (recommended for training)

# Train the model
model2.train(data='/content/DS_TFE_Final', epochs=20)

100%|██████████| 5.30M/5.30M [00:00<00:00, 66.5MB/s]


Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/DS_TFE_Final, epochs=20, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

100%|██████████| 6.23M/6.23M [00:00<00:00, 75.1MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /content/DS_TFE_Final/train... 5432 images, 0 corrupt: 100%|██████████| 5432/5432 [00:10<00:00, 519.37it/s]

...

                   all      0.813          1



20 epochs completed in 2.337 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1442566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 25/25 [00:49<00:00,  1.97s/it]


                   all      0.823          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9d90db3a30>
curves: []
curves_results: []
fitness: 0.9116128981113434
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8232257962226868, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9116128981113434}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.10279132473853327, 'inference': 0.30679302830849925, 'loss': 0.0004531491187310988, 'postprocess': 0.0004860662644909274}
task: 'classify'
top1: 0.8232257962226868
top5: 1.0

In [ ]:
results2_val = model2.val()

print(results2_val)

Ultralytics YOLOv8.2.24 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


val: Scanning /content/DS_TFE_Final/val... 775 images, 0 corrupt: 100%|██████████| 775/775 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


                   all      0.825          1
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train4


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9d98836e30>
curves: []
curves_results: []
fitness: 0.9122580587863922
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8245161175727844, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9122580587863922}
save_dir: PosixPath('runs/classify/train4')
speed: {'preprocess': 0.08026830611690398, 'inference': 0.6786275679065336, 'loss': 0.0009062982374621976, 'postprocess': 0.000934600830078125}
task: 'classify'
top1: 0.8245161175727844
top5: 1.0


In [ ]:
from ultralytics import YOLO

# Cargar el modelo pre-entrenado
model3 = YOLO('yolov8n-cls.pt')

# Establecer parámetros de entrenamiento
training_params = {
    'data': '/content/DS_TFE_Final',
    'epochs': 20,      # Cambia según tus necesidades
    'batch': 20,       # Tamaño del lote
    'imgsz': 224,      # Tamaño de imagen
    'optimizer': 'Adam',  # Optimizador
    'lr0': 0.0001,       # Tasa de aprendizaje
}

# Entrenar el modelo
model3.train(**training_params)

100%|██████████| 5.30M/5.30M [00:00<00:00, 69.4MB/s]


Ultralytics YOLOv8.2.25 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/DS_TFE_Final, epochs=20, time=None, patience=100, batch=20, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning /content/DS_TFE_Final/train... 5432 images, 0 corrupt: 100%|██████████| 5432/5432 [00:01<00:00, 2856.29it/s]


train: New cache created: /content/DS_TFE_Final/train.cache


val: Scanning /content/DS_TFE_Final/val... 775 images, 0 corrupt: 100%|██████████| 775/775 [00:00<00:00, 2752.23it/s]

...

                   all       0.81          1



20 epochs completed in 2.457 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.25 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
YOLOv8n-cls summary (fused): 73 layers, 1442566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 20/20 [00:45<00:00,  2.29s/it]


                   all      0.806          1
Speed: 0.0ms preprocess, 7.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c5e3e4b39a0>
curves: []
curves_results: []
fitness: 0.9032258093357086
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8064516186714172, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9032258093357086}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.000518675773374496, 'inference': 7.490338356264176, 'loss': 6.18350121282762e-05, 'postprocess': 4.460734705771169e-05}
task: 'classify'
top1: 0.8064516186714172
top5: 1.0

In [ ]:
results3_val = model3.val()

print(results3_val)

Ultralytics YOLOv8.2.25 🚀 Python-3.10.12 torch-2.3.0+cpu CPU (Intel Xeon 2.00GHz)
YOLOv8n-cls summary (fused): 73 layers, 1442566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/DS_TFE_Final/train... found 5432 images in 6 classes ✅ 
val: /content/DS_TFE_Final/val... found 775 images in 6 classes ✅ 
test: /content/DS_TFE_Final/test... found 1554 images in 6 classes ✅ 


val: Scanning /content/DS_TFE_Final/val... 775 images, 0 corrupt: 100%|██████████| 775/775 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 39/39 [00:46<00:00,  1.18s/it]


                   all      0.806          1
Speed: 0.0ms preprocess, 7.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c5e27f11cc0>
curves: []
curves_results: []
fitness: 0.9032258093357086
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8064516186714172, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9032258093357086}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.0009038371424521169, 'inference': 7.249378696564705, 'loss': 0.0001033659904233871, 'postprocess': 8.767650973412299e-05}
task: 'classify'
top1: 0.8064516186714172
top5: 1.0
